In [1]:
import cv2
import os
import numpy as np
from tqdm import tqdm

In [2]:
frames_path = r"C:\\Users\\simar\\OneDrive\\Desktop\\Python_stabilization\\unzipped_video"
video_name = r"C:\\Users\\simar\\OneDrive\\Desktop\\Python_stabilization\\DJI_20250411113208_0019_D.MP4"


In [3]:
frames = sorted(os.listdir(frames_path))
frame_paths = [os.path.join(frames_path, f) for f in frames]

sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

homographies = []

for i in tqdm(range(len(frame_paths) - 1)):
    img1 = cv2.imread(frame_paths[i], cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(frame_paths[i + 1], cv2.IMREAD_GRAYSCALE)
    if img1 is None or img2 is None:
        continue

    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    if des1 is None or des2 is None or len(kp1) < 4 or len(kp2) < 4:
        continue

    matches = bf.knnMatch(des1, des2, k=2)

    # Apply Lowe’s ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) < 4:
        continue

    pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    H, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, 5.0)
    if H is not None:
        homographies.append(H)

100%|██████████| 7178/7178 [3:28:29<00:00,  1.74s/it]     


In [9]:
import cvxpy as cp
import numpy as np

def smooth_full_homography_params(homographies, smoothness=50):
    """
    Smooth all 8 parameters of each homography using L1-norm minimization.
    Returns a list of smoothed homographies.
    """
    params = np.array([
        [H[0, 0], H[0, 1], H[0, 2],
         H[1, 0], H[1, 1], H[1, 2],
         H[2, 0], H[2, 1]] for H in homographies
    ])
    
    smoothed_params = []

    for i in range(8):  # For each of the 8 parameters
        p = params[:, i]
        n = len(p)
        x = cp.Variable(n)
        obj = cp.Minimize(cp.norm(x - p, 1) + smoothness * cp.norm1(cp.diff(x, 2)))
        cp.Problem(obj).solve()
        smoothed_params.append(x.value)

    smoothed_params = np.array(smoothed_params).T

    smoothed_H = []
    for row in smoothed_params:
        H = np.array([
            [row[0], row[1], row[2]],
            [row[3], row[4], row[5]],
            [row[6], row[7], 1.0]
        ])
        smoothed_H.append(H)

    return smoothed_H


In [12]:
# Define video settings
video_name = r"C:\Users\simar\OneDrive\Desktop\Python_stabilization\DJI_20250411113208_0019_D.MP4"
cap = cv2.VideoCapture(video_name)
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Video FPS: {fps}")

# Get frame size
# Find the first valid image to extract size
sample_frame = None
for path in frame_paths:
    sample_frame = cv2.imread(path)
    if sample_frame is not None:
        break

if sample_frame is None:
    raise ValueError("❌ Could not load any valid image frame from frame_paths.")

h, w = sample_frame.shape[:2]


# Output video writer
out_path = "2SIFT_RANSAC_Homography_L1_Stabilized.mp4"
out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# Warp and write frames
for i in tqdm(range(len(smoothed_H))):
    frame = cv2.imread(frame_paths[i])
    if frame is None:
        continue

    H = smoothed_H[i]
    stabilized = cv2.warpPerspective(frame, H, (w, h))  # use warpPerspective for Homography
    out.write(stabilized)

# Optionally write the last frame
last_frame = cv2.imread(frame_paths[-1])
if last_frame is not None:
    out.write(last_frame)

out.release()
print("✅ Stabilized video saved as:", out_path)


Video FPS: 29.97002997002997


  0%|          | 0/7168 [00:00<?, ?it/s]

100%|██████████| 7168/7168 [02:33<00:00, 46.84it/s]

✅ Stabilized video saved as: 2SIFT_RANSAC_Homography_L1_Stabilized.mp4


In [7]:
print(frame_paths[0])


C:\\Users\\simar\\OneDrive\\Desktop\\Python_stabilization\\unzipped_video\+Ransac.ipynb
